In [1]:
import random
import torch
import torchmetrics

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [3]:
# Hyperparameters

batch_size = 32
initial_lr = 0.0008  
base_lr = 0.0008  
warmup_steps = 10 
epochs = 30

In [4]:
train_features = np.load("../data/processed/balance_16k/model_input/train_features.npy")
train_labels = np.load("../data/processed/balance_16k/model_input/train_labels.npy")

test_features = np.load("../data/processed/balance_16k/model_input/test_features.npy")
test_labels = np.load("../data/processed/balance_16k/model_input/test_labels.npy")


In [5]:
train_features = torch.from_numpy(train_features).float()
train_features = train_features.to(device)
train_labels = torch.from_numpy(train_labels)
train_labels = train_labels.to(device)

test_features = torch.from_numpy(test_features).float()
test_features = test_features.to(device)
test_labels = torch.from_numpy(test_labels)
test_labels = test_labels.to(device)

In [6]:
print(f"Train features: {train_features.shape}, Train label: {train_labels.shape} ")
print(f"Test features: {test_features.shape}, Test label: {test_labels.shape} ")
train_min, train_max = train_features.min(), train_features.max()
print((train_min, train_max))

# inputs = 2 * (inputs - x_min) / (x_max - x_min) -1

Train features: torch.Size([14672, 3, 50, 50]), Train label: torch.Size([14672]) 
Test features: torch.Size([1631, 3, 50, 50]), Test label: torch.Size([1631]) 
(tensor(0., device='cuda:0'), tensor(13373320., device='cuda:0'))


In [7]:
dataset = TensorDataset(train_features, train_labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2, padding=1)
        # self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=2, padding=1
        )
        # self.bn2 = nn.BatchNorm2d(64)  # Add BatchNorm after the second conv layer

        self.conv3 = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=2, padding=1
        )

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # self.bn3 = nn.BatchNorm2d(128)  # Add BatchNorm after the third conv layer

        # self.dropout = nn.Dropout(p=0.25)

        self.fc1 = nn.Linear(128 * 7 * 7, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn_fc2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 6)

    def forward(self, x):
        x = self.pool(F.leaky_relu((self.conv1(x))))  # Conv1 -> -> leaky_relu -> Pool
        x = self.pool(F.tanh((self.conv2(x))))  # Conv2 -> BN -> Tanh -> Pool
        x = self.pool(F.tanh((self.conv3(x))))  # Conv3 -> BN -> Tanh -> Pool
        x = x.view(-1, 128 * 7 * 7)
        x = F.tanh(self.bn_fc1(self.fc1(x)))  # FC1 -> BN -> Tanh
        x = F.tanh(self.bn_fc2(self.fc2(x)))  # FC2 -> BN -> Tanh
        x = self.fc3(x)
        return x

In [9]:
model = CNN()
model = model.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Rprop(model.parameters(), lr=0.0008)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


for epoch in range(epochs):
    p_lr = ""
    
    if epoch < warmup_steps:
        lr = initial_lr + (base_lr - initial_lr) * (epoch / warmup_steps)
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
        # print(f"Warmup Epoch {epoch+1}/{warmup_steps}, Learning Rate: {lr:.8f}")
    else:
        # After warmup, allow the scheduler to control the learning rate
        scheduler.step()  # Step the scheduler after each epoch
        for param_group in optimizer.param_groups:
            lr = param_group["lr"]  # Get the current learning rate from the scheduler
    # print(f"Epoch {epoch+1}/{epochs}, Learning Rate: {lr:.8f}")

    batch_loss = 0.0
    for batch_inputs, batch_targets in dataloader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()

        batch_loss += loss.item()

    # Calculate average loss for the epoch
    avg_loss = batch_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Cross Entropy Loss: {avg_loss:.4f},Learningrate: {lr: 0.6f}")

Epoch 1/30, Cross Entropy Loss: 1.2336,Learningrate:  0.000800
Epoch 2/30, Cross Entropy Loss: 1.1718,Learningrate:  0.000800
Epoch 3/30, Cross Entropy Loss: 1.1443,Learningrate:  0.000800
Epoch 4/30, Cross Entropy Loss: 1.1192,Learningrate:  0.000800
Epoch 5/30, Cross Entropy Loss: 1.0958,Learningrate:  0.000800
Epoch 6/30, Cross Entropy Loss: 1.0765,Learningrate:  0.000800
Epoch 7/30, Cross Entropy Loss: 1.0565,Learningrate:  0.000800
Epoch 8/30, Cross Entropy Loss: 1.0385,Learningrate:  0.000800
Epoch 9/30, Cross Entropy Loss: 1.0197,Learningrate:  0.000800
Epoch 10/30, Cross Entropy Loss: 1.0043,Learningrate:  0.000800
Epoch 11/30, Cross Entropy Loss: 0.9889,Learningrate:  0.000800
Epoch 12/30, Cross Entropy Loss: 0.9795,Learningrate:  0.000800
Epoch 13/30, Cross Entropy Loss: 0.9617,Learningrate:  0.000800
Epoch 14/30, Cross Entropy Loss: 0.9492,Learningrate:  0.000800
Epoch 15/30, Cross Entropy Loss: 0.9388,Learningrate:  0.000800
Epoch 16/30, Cross Entropy Loss: 0.9250,Learningr

In [10]:
model.eval()

test_dataset = TensorDataset(test_features, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=6).to(device)

with torch.no_grad():
    for batch_test_inputs, batch_test_targets in test_dataloader:
        outputs = model(batch_test_inputs)
        predictions = torch.argmax(outputs, dim=1)
        accuracy.update(predictions, batch_test_targets)

final_accuracy = accuracy.compute().item()
print(f"Accuracy on the test data: {final_accuracy * 100:.2f}%")

accuracy.reset()

Accuracy on the test data: 68.55%
